In [1]:
import pandas as pd
import os
from openfisca_france_indirect_taxation.utils import assets_directory

In [9]:
from openfisca_survey_manager.coicop import build_raw_coicop_nomenclature

In [2]:
from openfisca_france_indirect_taxation.scripts.build_coicop_nomenclature import build_complete_coicop_nomenclature

In [5]:
build_complete_coicop_nomenclature(year = 1998, to_csv = True)

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
0,Boissons alcoolisées et tabac,Tabac,Cigares et cigarillos,Cigares et cigarillos,Cigares et cigarillos,02.2.1
1,Boissons alcoolisées et tabac,Tabac,Cigarettes,Cigarettes,Cigarettes,02.2.2
2,Boissons alcoolisées et tabac,Tabac,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,02.2.3
3,Boissons alcoolisées et tabac,Tabac,Stupéfiants,Stupéfiants,Stupéfiants,02.3
4,Produits alimentaires et boissons non alcoolisées,Produits alimentaires,Pain et céréales,Pain,Pain,1.1.1.1.1
...,...,...,...,...,...,...
304,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Articles à base de papier,Autres articles imprimés,9.5.3.1.1
305,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Articles à base de papier,Articles de papeterie,9.5.3.1.2
306,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Autres articles de bureau,"Stylos, crayons et encres",9.5.3.2.1
307,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Autres articles de bureau,Autres fournitures scolaires et de bureau,9.5.3.2.2


In [5]:
legislation_directory = os.path.join(
    assets_directory,
    'legislation',
    )

In [ ]:
def build_complete_coicop_nomenclature_new(year = 2016,to_csv=True):
    """Build a complete COICOP nomenclature including extra manually defined items."""
    
    # Start with the raw nomenclature
    coicop_nomenclature = build_raw_coicop_nomenclature(year)
    if year == 1998:
        # Extra items to add
        items = [
            ('Cigares et cigarillos', '02.2.1'),
            ('Cigarettes', '02.2.2'),
            ("Tabac sous d'autres formes et produits connexes", '02.2.3'),
            ('Stupéfiants', '02.3'),
        ]

        # Build a DataFrame for extra items in one go
        extra_data = pd.DataFrame(
            [{
                "label_division": "Boissons alcoolisées et tabac",
                "label_groupe": "Tabac",
                "label_classe": label,
                "label_sous_classe": label,
                "label_poste": label,
                "code_coicop": code
            } for label, code in items],
            dtype="str"
        )

        # Concatenate once and sort
        coicop_nomenclature = pd.concat([coicop_nomenclature, extra_data], ignore_index=True)
        coicop_nomenclature = coicop_nomenclature.sort_values("code_coicop").reset_index(drop=True)

    # Optionally save
    if to_csv:
        coicop_nomenclature.to_csv(
            os.path.join(legislation_directory, "nomenclature_coicop.csv"),
            index=False
        )
    
    return coicop_nomenclature[
        ["label_division", "label_groupe", "label_classe",
         "label_sous_classe", "label_poste", "code_coicop"]
    ].copy()


In [7]:
complete_new_1998 = build_complete_coicop_nomenclature_new(year = 1998, to_csv = False)

In [21]:
def build_complete_coicop_nomenclature_old(year = 2016,to_csv = True):
    coicop_nomenclature = build_raw_coicop_nomenclature(year)
    items = [
        ('Cigares et cigarillos', '02.2.1'),
        ('Cigarettes', '02.2.2'),
        ("Tabac sous d'autres formes et produits connexes", '02.2.3'),
        ('Stupéfiants ', '02.3'),
        ]

    for label_poste, code_coicop in items:
        label_division = 'Boissons alcoolisées et tabac'
        label_groupe = 'Tabac'
        label_classe = label_sous_classe = label_poste
        data = dict(
            label_division = [label_division],
            label_groupe = [label_groupe],
            label_classe = [label_classe],
            label_sous_classe = [label_sous_classe],
            label_poste = [label_poste],
            code_coicop = [code_coicop],
            )
        coicop_nomenclature = pd.concat(
            [coicop_nomenclature, pd.DataFrame.from_dict(data, dtype = 'str')],
            ignore_index = True
            )
        coicop_nomenclature.sort_values('code_coicop', inplace = True)

    if to_csv:
        coicop_nomenclature.to_csv(
            os.path.join(legislation_directory, 'nomenclature_coicop.csv'),
            )

    return coicop_nomenclature[['label_division', 'label_groupe', 'label_classe',
       'label_sous_classe', 'label_poste', 'code_coicop']].copy()

In [22]:
complete_old_1998 = build_complete_coicop_nomenclature_old(year = 1998, to_csv = False)

In [8]:
complete_1998 = build_complete_coicop_nomenclature(year = 1998, to_csv = False)

In [23]:
complete_old_1998.loc[complete_old_1998['label_groupe'] == 'Tabac']

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
0,Boissons alcoolisées et tabac,Tabac,Cigares et cigarillos,Cigares et cigarillos,Cigares et cigarillos,02.2.1
1,Boissons alcoolisées et tabac,Tabac,Cigarettes,Cigarettes,Cigarettes,02.2.2
2,Boissons alcoolisées et tabac,Tabac,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,02.2.3
308,Boissons alcoolisées et tabac,Tabac,Stupéfiants,Stupéfiants,Stupéfiants,02.3
128,Boissons alcoolisées et tabac,Tabac,Tabac,Tabac,Tabac,2.2.1.1.1


In [9]:
complete_1998.loc[complete_1998['label_groupe'] == 'Tabac']

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
0,Boissons alcoolisées et tabac,Tabac,Cigares et cigarillos,Cigares et cigarillos,Cigares et cigarillos,02.2.1
1,Boissons alcoolisées et tabac,Tabac,Cigarettes,Cigarettes,Cigarettes,02.2.2
2,Boissons alcoolisées et tabac,Tabac,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,02.2.3
3,Boissons alcoolisées et tabac,Tabac,Stupéfiants,Stupéfiants,Stupéfiants,02.3
129,Boissons alcoolisées et tabac,Tabac,Tabac,Tabac,Tabac,2.2.1.1.1


In [10]:
complete_new_1998.loc[complete_new_1998['label_groupe'] == 'Tabac']

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
0,Boissons alcoolisées et tabac,Tabac,Cigares et cigarillos,Cigares et cigarillos,Cigares et cigarillos,02.2.1
1,Boissons alcoolisées et tabac,Tabac,Cigarettes,Cigarettes,Cigarettes,02.2.2
2,Boissons alcoolisées et tabac,Tabac,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,02.2.3
3,Boissons alcoolisées et tabac,Tabac,Stupéfiants,Stupéfiants,Stupéfiants,02.3
129,Boissons alcoolisées et tabac,Tabac,Tabac,Tabac,Tabac,2.2.1.1.1


In [37]:
complete_new_1998

,label_division,label_groupe,label_classe,label_sous_classe,label_poste,code_coicop
0,Boissons alcoolisées et tabac,Tabac,Cigares et cigarillos,Cigares et cigarillos,Cigares et cigarillos,02.2.1
1,Boissons alcoolisées et tabac,Tabac,Cigarettes,Cigarettes,Cigarettes,02.2.2
2,Boissons alcoolisées et tabac,Tabac,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,Tabac sous d'autres formes et produits connexes,02.2.3
3,Boissons alcoolisées et tabac,Tabac,Stupéfiants,Stupéfiants,Stupéfiants,02.3
4,Produits alimentaires et boissons non alcoolisées,Produits alimentaires,Pain et céréales,Pain,Pain,1.1.1.1.1
...,...,...,...,...,...,...
304,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Articles à base de papier,Autres articles imprimés,9.5.3.1.1
305,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Articles à base de papier,Articles de papeterie,9.5.3.1.2
306,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Autres articles de bureau,"Stylos, crayons et encres",9.5.3.2.1
307,Loisirs et culture,"Presse, livres et papeterie","Imprimés divers, articles de papeterie et de d...",Autres articles de bureau,Autres fournitures scolaires et de bureau,9.5.3.2.2


In [52]:
complete_new_1998.label_division.unique()

array(['Boissons alcoolisées et tabac',
       'Produits alimentaires et boissons non alcoolisées', 'Éducation',
       'Hôtellerie, cafés, restauration', 'Autres biens et services',
       'Habillement et chaussures',
       'Logement, eau, gaz, électricité et autres combustibles',
       'Ameublement, équipement ménager et entretien courant de la maison',
       'Santé', 'Transport', 'Communications', 'Loisirs et culture'],
      dtype=object)

In [25]:
complete_new_2016 = build_complete_coicop_nomenclature_new(year = 2016, to_csv = False)

In [27]:
complete_new_2016.label_division.unique()

array(['Produits alimentaires et boissons non alcoolisées',
       'Boissons alcoolisées, tabac et stupéfiants',
       "Articles d'habillement et chaussures",
       'Logement, eau, gaz, électricité et autres combustibles',
       'Meubles, articles de ménage et entretien courant du foyer',
       'Santé', 'Transports', 'Communications', 'Loisirs et culture',
       'Enseignement', 'Restaurants et hôtels',
       'Biens et services divers'], dtype=object)